<a href="https://colab.research.google.com/github/yiwenwangANU/Machine_Learning/blob/main/ct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import urllib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds
import zipfile

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
train_data, test_data = fashion_mnist.load_data()
train_data[0]

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [ ]:
train_data = train_data.batch(32).prefetch(tf.data.AUTOTUNE)
test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
def windowed_dataset(series, batch_size, n_past=10, n_future=10, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
df = pd.read_csv('Weekly_U.S.Diesel_Retail_Prices.csv',
                     infer_datetime_format=True, index_col='Week of', header=0)

In [ ]:
df

,Weekly U.S. No 2 Diesel Retail Prices Dollars per Gallon
Week of,
1994-03-21,1.106
1994-03-28,1.107
1994-04-04,1.109
1994-04-11,1.108
1994-04-18,1.105
...,...
2021-05-31,3.255
2021-06-07,3.274
2021-06-14,3.286


In [ ]:
N_FEATURES = len(df.columns)
N_FEATURES

1

In [ ]:
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))
data

array([[0.03211587],
       [0.03232578],
       [0.03274559],
       ...,
       [0.48971453],
       [0.48992443],
       [0.49265323]])

In [ ]:
SPLIT_TIME = int(len(data) * 0.8) # DO NOT CHANGE THIS
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

In [ ]:
print(tf.reduce_mean(x_train))
print(tf.math.reduce_std(x_train))

tf.Tensor(0.27811785641359077, shape=(), dtype=float64)
tf.Tensor(0.22783185260989788, shape=(), dtype=float64)


In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

In [ ]:
BATCH_SIZE = 32  # ADVISED NOT TO CHANGE THIS
N_PAST = 10  # DO NOT CHANGE THIS
N_FUTURE = 10  # DO NOT CHANGE THIS
SHIFT = 1  # DO NOT CHANGE THIS

In [ ]:
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

In [ ]:
for past, future in train_set.take(1):
  print(past[1], future[1])
  print(past.shape)

tf.Tensor(
[[0.03232578]
 [0.03274559]
 [0.03253568]
 [0.03190596]
 [0.03211587]
 [0.03169605]
 [0.03106633]
 [0.03064652]
 [0.03064652]
 [0.03043661]], shape=(10, 1), dtype=float64) tf.Tensor(
[[0.03106633]
 [0.03043661]
 [0.03148615]
 [0.03253568]
 [0.03274559]
 [0.0329555 ]
 [0.03316541]
 [0.03316541]
 [0.03421495]
 [0.03652393]], shape=(10, 1), dtype=float64)
(32, 10, 1)


In [ ]:
model = tf.keras.models.Sequential([
        layers.Input(shape=(10, 1)),
        layers.Bidirectional(layers.LSTM(128, activation="relu", return_sequences=True)),
        layers.LSTM(128, activation="relu", return_sequences=True),
        layers.Dense(32, activation="relu"),                            
        tf.keras.layers.Dense(N_FEATURES)
    ])


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_4 (Bidirectio  (None, 10, 256)          133120    
 nal)                                                            
                                                                 
 lstm_15 (LSTM)              (None, 10, 128)           197120    
                                                                 
 dense_14 (Dense)            (None, 10, 32)            4128      
                                                                 
 dense_15 (Dense)            (None, 10, 1)             33        
                                                                 
Total params: 334,401
Trainable params: 334,401
Non-trainable params: 0
_________________________________________________________________


In [ ]:
inputs = layers.Input(shape=(10, 1))
#x = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(inputs) # expand input dimension to be compatible with LSTM
print(inputs.shape)
x = layers.Bidirectional(layers.LSTM(128, activation="relu", return_sequences=True))(inputs) # this layer will error if the inputs are not the right shape
x = layers.LSTM(128, activation="relu", return_sequences=True)(x) # using the tanh loss function results in a massive error
# print(x.shape)
# Add another optional dense layer (you could add more of these to see if they improve model performance)
x = layers.Dense(32, activation="relu")(x)
output = layers.Dense(N_FEATURES)(x)
print(output.shape)
model_lstm = tf.keras.Model(inputs=inputs, outputs=output, name="model_lstm")

(None, 10, 1)


(None, 10, 1)


In [ ]:


model_lstm.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam())



In [ ]:


model_lstm.fit(train_set,
            epochs=100,
            verbose=1,
            batch_size=128,
            validation_data=valid_set,
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="loss",
                                                          patience=10,
                                                          restore_best_weights=True),
                         tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",
                                                              patience=5,
                                                              verbose=1)])



In [ ]:
# COMPLETE THE CODE IN THIS FUNCTION
def solution_model():
    # DO NOT CHANGE THIS CODE
    # Reads the dataset.
    df = pd.read_csv('Weekly_U.S.Diesel_Retail_Prices.csv',
                     infer_datetime_format=True, index_col='Week of', header=0)

    # Number of features in the dataset. We use all features as predictors to
    # predict all features of future time steps.
    N_FEATURES = len(df.columns) # DO NOT CHANGE THIS

    # Normalizes the data
    data = df.values
    data = normalize_series(data, data.min(axis=0), data.max(axis=0))

    # Splits the data into training and validation sets.
    SPLIT_TIME = int(len(data) * 0.8) # DO NOT CHANGE THIS
    x_train = data[:SPLIT_TIME]
    x_valid = data[SPLIT_TIME:]

    # DO NOT CHANGE THIS CODE
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)

    # DO NOT CHANGE BATCH_SIZE IF YOU ARE USING STATEFUL LSTM/RNN/GRU.
    # THE TEST WILL FAIL TO GRADE YOUR SCORE IN SUCH CASES.
    # In other cases, it is advised not to change the batch size since it
    # might affect your final scores. While setting it to a lower size
    # might not do any harm, higher sizes might affect your scores.
    BATCH_SIZE = 32  # ADVISED NOT TO CHANGE THIS

    # DO NOT CHANGE N_PAST, N_FUTURE, SHIFT. The tests will fail to run
    # on the server.
    # Number of past time steps based on which future observations should be
    # predicted
    N_PAST = 10  # DO NOT CHANGE THIS

    # Number of future time steps which are to be predicted.
    N_FUTURE = 10  # DO NOT CHANGE THIS

    # By how many positions the window slides to create a new window
    # of observations.
    SHIFT = 1  # DO NOT CHANGE THIS

    # Code to create windowed train and validation datasets.
    train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
    valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

    # Code to define your model.
    model = tf.keras.models.Sequential([

        # ADD YOUR LAYERS HERE.

        # If you don't follow the instructions in the following comments,
        # tests will fail to grade your code:
        # The input layer of your model must have an input shape of:
        # (BATCH_SIZE, N_PAST = 10, N_FEATURES = 1)
        # The model must have an output shape of:
        # (BATCH_SIZE, N_FUTURE = 10, N_FEATURES = 1).
        # Make sure that there are N_FEATURES = 1 neurons in the final dense
        # layer since the model predicts 1 feature.

        # HINT: Bidirectional LSTMs may help boost your score. This is only a
        # suggestion.

        # WARNING: If you are using the GRU layer, it is advised not to use the
        # recurrent_dropout argument (you can alternatively set it to 0),
        # since it has not been implemented in the cuDNN kernel and may
        # result in much longer training times.
        tf.keras.layers.Dense(N_FEATURES)
    ])

    # Code to train and compile the model
    optimizer =  # YOUR CODE HERE
    model.compile(
        # YOUR CODE HERE
    )
    model.fit(
        # YOUR CODE HERE
    )

    return model